# Coursera Capstone Week 3 Notebook

## Purpose

Classify similar neighbourhoods in Toronto into 5 Clusters

## Data

The Toronto data requried for this analysis can be found at:
`https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M`



### Import all required libraries

In [10]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
#from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

# BeautifulSoup reqd to parse HTML from Wikipedia
#import bs4 as bs4
from bs4 import BeautifulSoup

print('Libraries imported.')



Libraries imported.


### Read Wiki Page

In [118]:
# create URL
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'

page = requests.get(url)
print(page)
print("Content-type:" + page.headers['content-type'])



<Response [200]>
Content-type:text/html; charset=UTF-8


### Parse HTML Wikimeadia page to extract the Table of postcodes

In [117]:
soup = BeautifulSoup(page.text, 'html.parser')
#soup.prettify()
table = soup.find('table', {'class': "wikitable sortable"})
tbody = table.find('tbody')
tr = tbody.findNext('tr')

### Sample code
#for anchors in soup.findAll('a', href=True):
#    print(anchors)
    
#>>> from bs4 import BeautifulSoup
#>>> soup = BeautifulSoup(unicodestring_containing_the_entire_htlm_doc)
#>>> table = soup.find('table', {'class': 'details'})
#>>> th = table.find('th', text='Issued on:')
#>>> th
#<th>Issued on:</th>
#>>> td = th.findNext('td')
#>>> td
#<td>2013-06-13</td>
#>>> td.text
#u'2013-06-13'    


### Create Dataframes

In [106]:
html_tables = pd.read_html(url)
toronto_pd = html_tables[0]
toronto_pd.columns = ['PostalCode','Borough','Neighbourhood']
toronto_pd.drop(toronto_pd.index[0], inplace=True) #Drorp the title row
print(toronto_pd.shape)
toronto_pd.head()

(289, 3)


,PostalCode,Borough,Neighbourhood
1,M1A,Not assigned,Not assigned
2,M2A,Not assigned,Not assigned
3,M3A,North York,Parkwoods
4,M4A,North York,Victoria Village
5,M5A,Downtown Toronto,Harbourfront


### Drop all rows with "Not assigned" Borough

In [113]:
toronto_pd = toronto_pd[toronto_pd.Borough != 'Not assigned']
print(toronto_pd.shape)
toronto_pd.head()

(212, 3)


,PostalCode,Borough,Neighbourhood
3,M3A,North York,Parkwoods
4,M4A,North York,Victoria Village
5,M5A,Downtown Toronto,Harbourfront
6,M5A,Downtown Toronto,Regent Park
7,M6A,North York,Lawrence Heights


### Replace "Not assigned" with Borough

In [166]:
toronto_pd.Neighbourhood = np.where(toronto_pd.Neighbourhood == 'Not assigned', toronto_pd.Borough,toronto_pd.Neighbourhood)
print(toronto_pd.shape)
toronto_pd.head(10)


(212, 3)


,PostalCode,Borough,Neighbourhood
3,M3A,North York,Parkwoods
4,M4A,North York,Victoria Village
5,M5A,Downtown Toronto,Harbourfront
6,M5A,Downtown Toronto,Regent Park
7,M6A,North York,Lawrence Heights
8,M6A,North York,Lawrence Manor
9,M7A,Queen's Park,Queen's Park
11,M9A,Etobicoke,Islington Avenue
12,M1B,Scarborough,Rouge
13,M1B,Scarborough,Malvern


### Compress Boroughs

Concatenate neighbourhoods in the same borough separating them with commas.

In [175]:
# Remove dupliates and concatenate neighbourhoods
u_toronto_pd = toronto_pd.groupby(['PostalCode','Borough'])['Neighbourhood'].apply(','.join).reset_index()
print(u_toronto_pd.shape)
u_toronto_pd.head()

(103, 3)


,PostalCode,Borough,Neighbourhood
0,M1B,Scarborough,"Rouge,Malvern"
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union"
2,M1E,Scarborough,"Guildwood,Morningside,West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [176]:
print("Sample")
u_toronto_pd[u_toronto_pd.PostalCode=="M9V"]

Sample


,PostalCode,Borough,Neighbourhood
101,M9V,Etobicoke,"Albion Gardens,Beaumond Heights,Humbergate,Jam..."


In [177]:
u_toronto_pd.shape

(103, 3)

### Read Geospatial data from file

In [178]:
latlng = pd.read_csv("http://cocl.us/Geospatial_data")
latlng.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


### Merge with Geo Data

In [181]:
geo_toronto_pd = u_toronto_pd.set_index('PostalCode').join(latlng.set_index('Postal Code'))
geo_toronto_pd.head(10)

,Borough,Neighbourhood,Latitude,Longitude
PostalCode,,,,
M1B,Scarborough,"Rouge,Malvern",43.806686,-79.194353
M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union",43.784535,-79.160497
M1E,Scarborough,"Guildwood,Morningside,West Hill",43.763573,-79.188711
M1G,Scarborough,Woburn,43.770992,-79.216917
M1H,Scarborough,Cedarbrae,43.773136,-79.239476
M1J,Scarborough,Scarborough Village,43.744734,-79.239476
M1K,Scarborough,"East Birchmount Park,Ionview,Kennedy Park",43.727929,-79.262029
M1L,Scarborough,"Clairlea,Golden Mile,Oakridge",43.711112,-79.284577
M1M,Scarborough,"Cliffcrest,Cliffside,Scarborough Village West",43.716316,-79.239476
